In [ ]:
from fpylll import IntegerMatrix, SVP, LLL
import numpy as np
import sympy as sp
from math import sqrt

In [ ]:
# find l^2-SVP of lattice given an oracle to find l-HSVP
def SVP (B, i):
    if(B.shape[1] == 1):
        return np.linalg.norm(B) #if there is just one basis vector
    
    [X, X_v] = HSVP(B, 0)
    det = np.linalg.det(B.T.dot(B))
    D = B.dot(np.linalg.inv(B.T.dot(B))) #dual lattice
    D_scaled = sqrt(det)*D
    [Y, Y_v] = HSVP(np.around(D_scaled), sqrt(det))
    L0 = sublatticeOrthogonalToGivenVector(np.around(D_scaled), Y_v, i)

    return min(np.linalg.norm(X), SVP(L0, i+1))

In [ ]:
#return the HSVP length and vector for the given lattice
def HSVP (B, d):
    B = np.matrix.tolist(B)
    B = [list(map(int, i)) for i in B]
    B = IntegerMatrix.from_matrix(B)
    L = LLL.reduction(B.transpose())

    L = [list(i) for i in L]

    if d:
        for i in range(len(L)):
            for j in range(len(L[0])):
                L[i][j] = L[i][j]/d

    res_h = 0
    for j in range(len(L[0])):
        res_h = res_h + L[0][j]*L[0][j]
    res_h = sqrt(res_h)
    return [res_h, L[0]]

In [ ]:
# return sublattice along the hyperplane orthogonal to Y
def sublatticeOrthogonalToGivenVector(D, Y, i):
    Y = np.matrix(Y)
    Y = Y.T
    n = D.shape[0]

    # vector which satifies DV=Y
    D_red = sp.Matrix(D)
    D_red = np.matrix(D_red.rref()[0], dtype='float')
    D_red = D_red[:n-i, :]
    Y_red = sp.Matrix(Y)
    Y_red = np.matrix(Y_red.rref()[0], dtype='float')
    Y_red = Y_red[:n-i, :]

    V = np.linalg.solve(D_red, Y_red)

    # this will store the unimodular matrix 
    # which when multiplied with D given a lattice basis which has first vector as Y
    U = np.zeros((D.shape[1], D.shape[1]))
    U[:,0] = V.T

    # dimension of U
    n = U.shape[1]

    # this will store the elementary operations matrix
    Elem = np.identity(n)

    # swapping first and last columns and transposing
    U = U.dot(swap(0, -1, n))
    U_T = U.T

    # columnGCD operations for all consecutive columns
    for i in range(n-1):
        [U, Elem] = columnGCD(i, i+1, U, Elem)

    I = np.identity(n)
    # inverting all the elementary operations
    # this creates a unimodular matrix whose first column has values 
    # such that on multiplying with D (dual of original) it will give 
    # a basis whose first column is Y
    U = I.dot(np.linalg.inv(Elem))
    U = I.T
    U = I.dot(swap(0, -1, n))

    # lattice basis of dual lattice D with first basis vector as Y
    # this has dimensions n x n-1
    M = D.dot(U)
    M = M.dot(np.linalg.inv(M.dot(M.T)))[:, 1:]

    return M

In [ ]:
# perform gcd operations on the last row elemets of the given vectors
# and return the matrix for elementary operations involved
def columnGCD(i, j, U, Elem):
    a = U[-1][i]
    b = U[-1][j]

    if(a==0):
        return [U, Elem]
    
    if(a>b):
        Elem = Elem.dot(swap(i, j, U.shape[0]))
        [U, Elem] = columnGCD(i, j, U.dot(Elem), Elem)
    else :
        Elem = Elem.dot(add(j, i, (-1)*(b/a), U.shape[0]))
        [U, Elem] = columnGCD(i, j, U.dot(Elem), Elem)

    return [U, Elem]


In [ ]:
def swap(i, j, n):
    I = np.identity(n)
    I[i][i] = I[j][j] = 0
    I[i][j] = I[j][i] = 1

    return I

In [ ]:
def invert(i, n):
    I = np.identity(n)
    I[i][i] = -1

    return I

In [ ]:
def add(i, j, c, n):
    I = np.identity(n)
    I[i][j] = c

    return I

In [ ]:
matrix_imp = IntegerMatrix.random(10, "uniform", bits=10)
print(matrix_imp)

with open("/path/to/file/to/store/matrix/t1.txt", 'w') as f:
    f.write(str(matrix_imp))

In [ ]:
fi = open("/path/to/file/to/store/matrix/t1.txt", 'r')
a = []
for l in fi:
    l = l.strip()
    l = l[1:-1]
    a.append([int (i) for i in l.split()])
fi.close()

B = np.matrix(a)
print(SVP(B, 0))